In [6]:
import os
import numpy as np
import torch
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv
import torch.nn as nn
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
def coord_convert(x,y,z):
    
    phi = np.arctan2(y, x)
    theta = np.arctan2(np.sqrt(x ** 2 + y ** 2), z)
    eta = -np.log(np.tan(theta/2))    
    return phi, eta, z


def sector_splitter(df, n, m):
    
    phi_bins = np.linspace(-np.pi, np.pi, n+1)  # bins for phi angles
    eta_bins = np.linspace(-4.5, 4.5, m+1)  # bins for eta angles
        
    df_list = []
    for i in range(n):
        for j in range(m):
            
            phi_mask = (phi_bins[i] < df['phi']) & (df['phi'] < phi_bins[i+1])
            eta_mask = (eta_bins[j] < df['eta']) & (df['eta'] < eta_bins[j+1])
            
            df_list.append(df[(phi_mask & eta_mask)])
                    
    return df_list


def data_puller(datamax):
    
    directory = '/home/aaportel/teams/group-3/data/'
    folders = ['train_1/','train_2/','train_5/']

    data_cutoff = 0
    data = []
   
    for folder in folders:
       
        hit_files = sorted([f for f in os.listdir(directory + folder) if f.endswith('hits.csv')])   
        truth_files = sorted([f for f in os.listdir(directory + folder) if f.endswith('truth.csv')])   
        
        for hit_file, truth_file in zip(hit_files, truth_files):

            # read a CSV file into a DataFrame
            X = pd.read_csv(directory + folder + hit_file, usecols=['x','y','z'])
            Y = pd.read_csv(directory + folder + truth_file, usecols=['particle_id'])
            
            # calculate phi, eta, and z from x, y, and z
            phi, eta, z = coord_convert(X['x'], X['y'],X['z'])

            # create a new DataFrame with phi, theta, eta, z and particle_id columns
            df = pd.DataFrame({
                'phi': phi,
                'eta': eta,
                'z': z,
                'particle_id': Y['particle_id'] 
            })

            # splits dataframe into chunks based on detector geometry
            n, m = 8, 4
            df_list = sector_splitter(df, n, m)
            
            # appends elements of df_list into a mega list
            data.extend(df_list)
            
            data_cutoff += 1
            if data_cutoff > datamax:
                return data
            
    return data

In [19]:
class CustomDataset(Dataset):
    def __init__(self, dataframes, device):
        self.dataframes = dataframes
        self.device = device
        
    def __len__(self):
        return len(self.dataframes)
    
    def __getitem__(self, idx):

        df = self.dataframes[idx]
        
        input_data = df[['phi','eta','z']]
        output_data = df['particle_id']

        # Create a tensor of node features by stacking the columns of the input data
        node_features = torch.tensor(input_data.values).to(self.device)
        output_features = torch.tensor(output_data.values).to(self.device)

        phi = torch.from_numpy(input_data[['phi']].values).to(self.device)
        eta = torch.from_numpy(input_data[['eta']].values).to(self.device)

        # Compute the pairwise differences between the phi and eta columns of adjacent nodes
        phi_diff = phi.unsqueeze(0) - phi.unsqueeze(-1)
        eta_diff = eta.unsqueeze(0) - eta.unsqueeze(-1)

        #this is deltaR
        diff_norm = torch.sqrt(phi_diff**2 + eta_diff**2)

        # Create a binary adjacency matrix based on a threshold of 1.7
        adjacency_matrix = torch.where(
            (diff_norm < 1.7) & (phi_diff > 0) & (eta_diff > 0), 
            torch.ones_like(diff_norm), 
            torch.zeros_like(diff_norm)
        )

        # Convert the adjacency matrix to a list of edge indices
        edge_indices = adjacency_matrix.squeeze(-1).nonzero(as_tuple=False).t()


        # Create a tensor of edge features by concatenating the phi and eta differences    
        phi_diff = phi_diff[edge_indices[0], edge_indices[1]]
        eta_diff = eta_diff[edge_indices[0], edge_indices[1]]
        edge_features = torch.cat((
            phi_diff.unsqueeze(-1),
            eta_diff.unsqueeze(-1)
        ), -1).to(self.device)

        # Create a PyTorch Geometric Data object
        data = Data(
            x          = node_features.float(), 
            edge_index = edge_indices.long().to(self.device), 
            edge_attr  = edge_features,
            y          = output_features
        )

        return data


In [20]:
class EdgeNetwork(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(EdgeNetwork, self).__init__()
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.mlp(x)

class NodeNetwork(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NodeNetwork, self).__init__()
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.mlp(x)

class EdgeConv(MessagePassing):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(EdgeConv, self).__init__(aggr='max')
        self.edge_mlp = EdgeNetwork(input_dim, hidden_dim, output_dim)
        self.node_mlp = NodeNetwork(input_dim+output_dim, hidden_dim, output_dim)

    def forward(self, x, edge_index, edge_attr):
        edge_feats = self.edge_mlp(edge_attr)
        self.edge_index = edge_index
        return self.propagate(edge_index, x=x, edge_feats=edge_feats)

    def message(self, x_j, edge_feats):
        return {'node_feats': torch.cat([x_j, edge_feats], dim=1)}

    def update(self, aggr_out, x):
        node_feats = self.node_mlp(aggr_out)
        return node_feats

class PFN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(PFN, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.edge_conv1 = EdgeConv(input_dim=2, hidden_dim=hidden_dim, output_dim=output_dim)
        self.edge_conv2 = EdgeConv(input_dim=output_dim*2, hidden_dim=hidden_dim, output_dim=output_dim)
        self.edge_conv3 = EdgeConv(input_dim=output_dim*2, hidden_dim=hidden_dim, output_dim=output_dim)
        self.edge_conv4 = EdgeConv(input_dim=output_dim*2, hidden_dim=hidden_dim, output_dim=output_dim)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        edge_attr = F.normalize(edge_attr, p=2, dim=1) # normalize edge attributes
        x = F.relu(self.edge_conv1(x, edge_index, edge_attr))
        x = F.relu(self.edge_conv2(x, edge_index, edge_attr))
        x = F.relu(self.edge_conv3(x, edge_index, edge_attr))
        x = F.relu(self.edge_conv4(x, edge_index, edge_attr))
        x = global_max_pool(x, data.batch)
        return F.log_softmax(x, dim=1)


In [21]:
dataframes = data_puller(1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = CustomDataset(dataframes, device=device)

# Split the dataset into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Define the model and optimizer
model = PFN(input_dim=3, hidden_dim=64, output_dim=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [16]:
# Train the model
for epoch in range(10):
    model.train()
    for data in train_dataset:
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        optimizer.step()

    # Evaluate the model on the test set
    model.eval()
    test_loss = 0.0
    correct = 0
    for data in test_dataset:
        output = model(data)
        test_loss += F.nll_loss(output, data.y, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(data.y.view_as(pred)).sum().item()

    test_loss /= len(test_dataset)
    accuracy = 100.0 * correct / len(test_dataset)

    print('Epoch: {:03d}, Test Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(
        epoch, test_loss, accuracy))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking arugment for argument mat2 in method wrapper_mm)